# Applying Advanced Transformations
Cameron Peace


## Task

<mark><u>**I. Clean the files and combine them into one final DataFrame.**

This dataframe should have the following columns:

* [ ] Hero (Just the name of the Hero)
* [ ] Publisher
* [ ] Gender
* [ ] Eye color
* [ ] Race
* [ ] Hair color
* [ ] Height (numeric)
* [ ] Skin color
* [ ] Alignment
* [ ] Weight (numeric)
* [ ] Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:
    * Agility
    * Flight
    * Superspeed
    * etc.

Hint: There is a space in "100 kg" or "52.5 cm"

<mark><u>**II. Use your combined DataFrame to answer the following questions.**

* [ ] Compare the average weight of super powers who have Super Speed to those who do not.
* [ ] What is the average height of heroes for each publisher?

## Data Dictionary

* **Row Identification** - index
* **name** 
* **Gender** 
* **Eye color** 
* **Race** 
* **Hair color** 
* **Height** - The height measured in centimeters
* **Publisher** - comics publisher
* **Skin color**
* **Alignment** - (good, bad, neutral, unknown)
* **Weight** - The weight measured in pounds

### Data Background

This dataset is a modified version of a dataset found [here](https://www.kaggle.com/datasets/claudiodavi/superhero-set) on kaggle.  It has been altered by Coding Dojo for educational purposes.

## Imports, Loading, Viewing Data

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [4]:
# loading the data
df_info = pd.read_csv('superhero_info.csv')
df_powers = pd.read_csv('superhero_powers.csv')

# making a copy
df_info_original = df_info.copy()
df_powers_original = df_powers.copy()

#### Viewing Superhero info df

In [8]:
# viewing superhero info
display(df_info.head())
display(df_info.info(), df_info.describe(), df_info.columns)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


None

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
count,463,463,463,463,463,463,463,463
unique,457,3,52,4,29,21,14,287
top,Spider-Man|Marvel Comics,Male,Human,good,Black,blue,Unknown,"{'Height': '183.0 cm', 'Weight': '90.0 kg'}"
freq,3,321,156,311,127,182,409,6


Index(['Hero|Publisher', 'Gender', 'Race', 'Alignment', 'Hair color',
       'Eye color', 'Skin color', 'Measurements'],
      dtype='object')

#### Viewing Superhero powers df

In [7]:
# viewing the data
display(df_powers.head())
display(df_powers.info(), df_powers.describe(), df_powers.columns)

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


None

,hero_names,Powers
count,667,667
unique,667,608
top,3-D Man,Intelligence
freq,1,8


Index(['hero_names', 'Powers'], dtype='object')

<mark><u>**Comment:**

<font color='dodgerblue' size=4><i>

* <font color='dodgerblue' size=4><i> 
Text
    * subtext
<font color='dodgerblue' size=4>
Text